<a href="https://colab.research.google.com/github/maktaurus/ML-Work/blob/main/Transfer_Learning_and_Fine_Tunning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

 **Transfer Learning**

A pre-trained model is a saved network that was previously trained on a large dataset, typically on a large-scale image-classification task.
The intuition behind transfer learning for image classification is that if a model is trained on a large and general enough dataset, this model will effectively serve as a generic model of the visual world. You can then take advantage of these learned feature maps without having to start from scratch by training a large model on a large dataset.


Import the model from keras

In [ ]:
from tensorflow.keras.applications.inception_v3 import InceptionV3

Then, Specifty the parameters. Use weights "imagenet" and input_shape of image for your desired problem.
Then freeze the layers so weights are not updated.


In [ ]:
base_model = InceptionV3(weights="imagenet", include_top=False, input_shape=(250,250,3))
base_model.trainable = False

In [ ]:
##Check the model summary
base_model.summary()


By pulling the summary of the model you can see that it is a very deep network. You can then select up to which point of the network you want to use. This is because the original last layer might be too specialized in what it has learned so it might not translate well into your application.


In [ ]:
input = tf.keras.layers.Input(shape=(None,None,3))
x = tf.keras.layers.RandomBrightness(0.2)(input)
x = base_model(x, training=True)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
output = tf.keras.layers.Dense(10, activation="softmax")(x)

model = tf.keras.Model(input, output)
model.summary()

Another way, where you want to select the layers from the network by specifying the names of layers.

In [ ]:
feature_maps = [layer.output for layer in base_model.layers if layer.name in ("activation_58","activation_59")]

OR check if the layer is isinstance of the layer class

In [ ]:
feature_maps = [layer.output for layer in base_model.layers if isinstance(layer, tf.keras.layers.Conv2D)]
feature_maps

In [ ]:
model = tf.keras.Model(base_model.input, feature_maps)
model.summary()

Another way, is by taking a specific layer in the network and taking its output as last layer in network. So the model will not extract much.

In [ ]:
extracted_layer = base_model.get_layer("activation_74").output
x = tf.keras.layers.GlobalAveragePooling2D()(extracted_layer)
output = tf.keras.layers.Dense(10, activation="softmax")(x)

model = tf.keras.Model(base_model.input, output)
model.summary()